In [1]:
import pandas as pd
import numpy as np
import re
import os, json

In [2]:
file_names = ["4146176 Raw Data.xlsx"]
templates = []
for files in file_names:
    print("###----###")
    if files.endswith(".xlsx") or files.endswith(".xls"):
        sheet_names = pd.ExcelFile(files).sheet_names
        print(f"Sheet names for {files} is {','.join(sheet_names)}")
        
        mapping_type_1, mapping_sheet_found, key_sheet_found = False, False, False

        check_for_curion = False
        for sheets in sheet_names:
            if "map" in sheets.lower():
                print(f"Mapping Sheet for {files} found with name {sheets}")

                mapping_sheet_found = True
                df = pd.read_excel(files, sheet_name=sheets)
                for col_names in df.columns:
                    if "variable" in col_names.lower() and "information" in col_names.lower():
                        mapping_type_1 = True

                df2 = pd.read_excel(files, sheet_name=sheets, header=None)
                for val in df2[0].to_list():
                    if "values" in str(val).lower():
                        check_for_curion = True
                        break

            elif "key" in sheets.lower():
                print(f"Key Sheet for {files} found with name {sheets}")
                key_sheet_found = True
            else:
                pass
        
        if mapping_sheet_found:
            if len(sheet_names) == 3 and mapping_type_1:
                template = "MARKET PROBE"
            elif len(sheet_names) == 2 and mapping_type_1:
                template = "IPSOS"
            elif len(sheet_names) == 2 and not mapping_type_1 and check_for_curion:
                template = "CURION"
            elif len(sheet_names) == 2 and not mapping_type_1 and not check_for_curion:
                template = "CS_CURION_CHECK"
            elif len(sheet_names) in [2, 3]:
                template = "CS_RAW_CHECK"
            else:
                template = "CS_RAW_OTHERS"
        elif key_sheet_found:
            if (len(sheet_names) in [2, 3]):
                template = "CS_RAW_KEY"
        else:
            template = "CS_RAW_OTHERS"
    else:
        template = "CS_RAW_NOT_EXCELS"

    templates.append({'file': files, 'template': template})



template_df = pd.DataFrame(templates)
display(template_df)

###----###
Sheet names for 4146176 Raw Data.xlsx is A1,Datamap
Mapping Sheet for 4146176 Raw Data.xlsx found with name Datamap


,file,template
0,4146176 Raw Data.xlsx,CURION


In [3]:
#processing questionnaires to get timepoint and other info

folder = "afr quess"
folder_path = os.path.abspath(folder)
all_files = os.listdir(folder_path)
json_files = [os.path.join(folder_path, f) for f in all_files if f.lower().endswith('.json')]

# Mapping of abbreviations to full words
unit_mapping = {
    "min": "minute",
    "mins": "minute",
    "hr": "hour",
    "hrs": "hour",
    "wk": "week",
    "wks": "week",
    "day": "day",
    "days": "day",
    "month": "month",
    "months": "month"
}

def extract_info(filename):
    study_numbers = re.findall(r'\b\d{7}\b', filename)
    literal_pattern = r'(?P<literal>immediate|overnight|baseline)'
    digit_first_pattern = (
        r'(?P<number>\d+)\s*(?:-?\s*)?'
        r'(?P<unit>(?:minute(?:s)?|min|hour(?:s)?|hr|day(?:s)?|month(?:s)?|week(?:s)?|wk))\b'
    )
    
    unit_first_pattern = (
        r'(?P<unit_first>(?:minute(?:s)?|min|hour(?:s)?|hr|day(?:s)?|month(?:s)?|week(?:s)?|wk))\b'
        r'\s*(?:-?\s*)?(?P<number_first>\d+)'
    )
    
    timepoint_pattern = re.compile(
        rf'(?i)({literal_pattern}|{digit_first_pattern}|{unit_first_pattern})'
    )
    
    timepoint = None
    match = timepoint_pattern.search(filename)
    if match:
        if match.group('literal'):
            timepoint = match.group('literal').lower()
        elif match.group('number') and match.group('unit'):
            unit = match.group('unit').lower()
            expanded_unit = unit_mapping.get(unit, unit) 
            timepoint = f"{match.group('number')} {expanded_unit}"
        elif match.group('unit_first') and match.group('number_first'):
            unit = match.group('unit_first').lower()
            expanded_unit = unit_mapping.get(unit, unit)
            timepoint = f"{match.group('number_first')} {expanded_unit}"

    is_regimen = "regimen" in filename.lower()
    is_screener = "screener" in filename.lower()

    return {
        'study_numbers': study_numbers,
        'timepoint': timepoint,
        'is_screener': is_screener,
        'is_regimen': is_regimen
    }

for file in json_files:
    result = extract_info(os.path.basename(file))
    print(result)


{'study_numbers': ['4146176', '4146177'], 'timepoint': '1 week', 'is_screener': False, 'is_regimen': False}
{'study_numbers': ['4146176', '4146177'], 'timepoint': '2 week', 'is_screener': False, 'is_regimen': False}
{'study_numbers': ['4146176', '4146177'], 'timepoint': '4 week', 'is_screener': False, 'is_regimen': False}
{'study_numbers': ['4146176', '4146177'], 'timepoint': 'immediate', 'is_screener': False, 'is_regimen': False}
{'study_numbers': ['4146176', '4146177'], 'timepoint': 'overnight', 'is_screener': False, 'is_regimen': False}
{'study_numbers': ['4146176', '4146177'], 'timepoint': '1 week', 'is_screener': False, 'is_regimen': True}
{'study_numbers': ['4146176'], 'timepoint': None, 'is_screener': True, 'is_regimen': False}


In [4]:
#function to get question number and question text from questionnaires
def question_number(s):
    pattern = r'^(?P<qnum>(?:q)?\d+(?:[a-z])?)\.\s*(?P<question>.*)$'
    match = re.match(pattern, s.strip(), re.IGNORECASE)
    if match:
        return match.group('qnum'), match.group('question').strip()
    return None, None

all_responses = []
for json_file in json_files:
    file_info = extract_info(os.path.basename(json_file))
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    content_data = data.get('analyzeResult', {}).get('paragraphs', [])
    
    i = 0
    while i < len(content_data):
        content = content_data[i].get('content', '').strip()
        ques_num, question_text = question_number(content)

        if ques_num:
            if not question_text and i + 1 < len(content_data):
                next_content = content_data[i + 1].get('content', '').strip()
                question_text = next_content
                i += 1

            all_responses.append({
                "Question_Number": ques_num,
                "Question": question_text,
                "Study_Number": ', '.join(file_info['study_numbers']) if file_info['study_numbers'] else "N/A",
                "Screener": file_info['is_screener'],
                "Regimen": file_info['is_regimen'],
                "Timepoint": file_info['timepoint'] if file_info['timepoint'] else "N/A"
            })
        
        i += 1
res_df = pd.DataFrame(all_responses)
res_df

,Question_Number,Question,Study_Number,Screener,Regimen,Timepoint
0,1a,Now that you have used the Eye Treatment for 1...,"4146176, 4146177",False,False,1 week
1,1b,What reasons do you have for making the choice...,"4146176, 4146177",False,False,1 week
2,2,"What, if anything, do you LIKE about the produ...","4146176, 4146177",False,False,1 week
3,3,"What, if anything, do you DISLIKE about the pr...","4146176, 4146177",False,False,1 week
4,4,How would you rate the product on an overall b...,"4146176, 4146177",False,False,1 week
...,...,...,...,...,...,...
106,18,Into which of the following categories does yo...,4146176,True,False,N/A
107,19a,"Are you of Hispanic, Latino, or Spanish origin...",4146176,True,False,N/A
108,19b,What is your race (origin)? SELECT ALL THAT APPLY,4146176,True,False,N/A
109,20,Now I would like you to think about the skin o...,4146176,True,False,N/A


In [5]:
def assign_timepoint_letters(df):
    unique_tps = [tp for tp in df['Timepoint'].dropna().unique() if tp != 'N/A']

    if any(df['Regimen'] == True):
        unique_tps.append('regimen')
    # to hours
    conversion_factors = {
        'week': 168,    
        'wk': 168,
        'day': 24,      
        'days': 24,
        'hour': 1,      
        'hr': 1,
        'minute': 1/60, 
        'min': 1/60,
        'minutes': 1/60,
        'year': 8760,  
        'years': 8760
    }
    
    def get_order(tp):
        tp_norm = tp.lower().strip()
        if tp_norm in ['immediate', 'baseline']:
            return 0
        elif tp_norm == 'overnight':
            return 1
        elif tp_norm == 'regimen':
            return 10000
        else:
            m = re.search(r'(\d+(?:\.\d+)?)\s*(week|wk|day|days|hour|hr|minute|min|minutes|year|years)', tp_norm)
            if m:
                num = float(m.group(1))
                unit = m.group(2)
                factor = conversion_factors.get(unit, None)
                if factor is not None:
                    return 2 + num * factor
            return 9999
    sorted_tps = sorted(unique_tps, key=get_order)
    mapping = {}
    for i, tp in enumerate(sorted_tps):
        mapping[chr(65 + i)] = tp
    return mapping

#Mapping data

In [6]:
def process_questionnaire(input_df, study_number, timepoints_letters):
    output = []
    current_main = None
    current_main_text = None
    current_values = []
    in_values_section = False
    got_sub_ques = False

    # Iterate over each row of the input DataFrame.
    for index, row in input_df.iterrows():
        # Clean each cell: fill NaN, convert to str and strip spaces.
        row = row.fillna('').astype(str).str.strip()
        
        # If first column exists and does not start with 'values' or 'value'
        if row[0] and not row[0].lower().startswith(('values', 'value')):
            # Flush any existing main if we are not in a values section.
            if not in_values_section and current_main and current_main_text is not None:
                output.append({
                    'Study Number': study_number,
                    'Main column': current_main,
                    'main_mean': current_main_text,
                    'sub column': '',
                    'sub_mean': '',
                    'values': '',
                    'meaning': ''
                })
            # If a colon is present, split into main column and its description.
            if ':' in row[0]:
                main_parts = row[0].split(':', 1)
                current_main = main_parts[0].strip()
                # Remove surrounding brackets if present.
                if current_main.startswith('[') and current_main.endswith(']'):
                    current_main = current_main[1:-1]
                current_main_text = main_parts[1].strip()
            else:
                current_main = row[0].strip()
                current_main_text = ''
            current_values = []
            in_values_section = False
            got_sub_ques = False

        # Identify start of the values section.
        elif row[0].lower().startswith(('values', 'value')):
            in_values_section = True      

        elif in_values_section:
            # If second column is a number (the value)
            if row[1] and row[1].isdigit():
                value = row[1]
                meaning = row[2] if row[2] else ''
                if value or meaning:
                    current_values.append((value, meaning))
            # If second column contains the sub question indicator (and is not just a digit)
            elif row[1] and current_main in row[1] and not row[1].isdigit():
                got_sub_ques = True
                sub_col = row[1]
                if sub_col.startswith('[') and sub_col.endswith(']'):
                    sub_col = sub_col[1:-1]
                sub_mean = row[2].strip() if row[2] else ''
                for val, mean in current_values:
                    output.append({
                        'Study Number': study_number,
                        'Main column': current_main,
                        'main_mean': current_main_text,
                        'sub column': sub_col,
                        'sub_mean': sub_mean,
                        'values': val,
                        'meaning': mean
                    })
            # If no sub question and blank second column, flush current values.
            elif (not row[1] and not got_sub_ques):
                if current_values:
                    for val, mean in current_values:
                        output.append({
                            'Study Number': study_number,
                            'Main column': current_main,
                            'main_mean': current_main_text,
                            'sub column': '',
                            'sub_mean': '',
                            'values': val,
                            'meaning': mean
                        })
                else:
                    output.append({
                        'Study Number': study_number,
                        'Main column': current_main,
                        'main_mean': current_main_text,
                        'sub column': '',
                        'sub_mean': '',
                        'values': '',
                        'meaning': ''
                    })
    
    # Final flush if no sub question was encountered.
    if not got_sub_ques:
        if current_values:
            for val, mean in current_values:
                output.append({
                    'Study Number': study_number,
                    'Main column': current_main,
                    'main_mean': current_main_text,
                    'sub column': '',
                    'sub_mean': '',
                    'values': val,
                    'meaning': mean
                })
        else:
            output.append({
                'Study Number': study_number,
                'Main column': current_main,
                'main_mean': current_main_text,
                'sub column': '',
                'sub_mean': '',
                'values': '',
                'meaning': ''
            })
    
    # Now, transform the intermediate output to the final format.
    final_rows = []
    for row in output:
        study_num = row['Study Number']
        # Variable: use sub column if available; else, main column.
        main_col = row['Main column'].strip() if row['Main column'] else ''
        sub_col  = row['sub column'].strip() if row['sub column'] else ''
        variable = sub_col if sub_col else main_col
        
        # Label: combine main_mean and sub_mean if both exist.
        main_mean = row['main_mean'].strip() if row['main_mean'] else ''
        sub_mean  = row['sub_mean'].strip() if row['sub_mean'] else ''
        if main_mean and sub_mean:
            label = f"{main_mean} {sub_mean}"
        else:
            label = main_mean if main_mean else sub_mean
        
        # MappedValue and MappedLabel.
        mapped_value = row['values']
        mapped_label = row['meaning']
        
        # Timepoint
        if variable.lower().startswith('s'):
            timepoint = "screener"
        else:            
            timepoint = timepoints_letters.get(variable[0].upper(),"Cannot be Mapped")
        
        # Quesion Number
        m = re.match(r'^[A-Za-z]+\s*(\d+)([a-zA-Z])?([a-zA-Z])?\s?', variable)
        if m:
            # If the first letter after the digits is 'r', ignore it.
            if m.group(2) and m.group(2).lower() == 'r':
                question_number = m.group(1)
            else:
                question_number = m.group(1) + (m.group(2) if m.group(2) else "")
        else:
            question_number = "" ''
        
        final_rows.append({
            "Study_Number": study_num,
            "Variable": variable,
            "Label": label,
            "MappedValue": mapped_value,
            "MappedLabel": mapped_label,
            "Timepoint": timepoint,
            "QuestionNumber": question_number
        })
    
    final_df = pd.DataFrame(final_rows, columns=[
        "Study_Number", "Variable", "Label", "MappedValue", "MappedLabel", "Timepoint", "QuestionNumber"
    ])
    return final_df

In [7]:
#merge questionnaire with mapping df

def merge_question_data(df_input, res_df):
    def study_match(study_str, study):
        studies = [s.strip() for s in study_str.split(',')]
        return study in studies

    def match_question(input_row):
        study = str(input_row['Study_Number']).strip()
        timepoint_input = str(input_row['Timepoint']).strip().lower()
        question_num = str(input_row['QuestionNumber']).strip().lower() if pd.notnull(input_row['QuestionNumber']) else None
        mask = res_df['Study_Number'].apply(lambda x: study_match(x, study))
        matching_study = res_df[mask]
        if matching_study.empty:
            return None

        if timepoint_input == "screener":
            filtered = matching_study[matching_study['Screener'] == True]
        elif timepoint_input == "regimen":
            filtered = matching_study[matching_study['Regimen'] == True]
        else:
            filtered = matching_study[
                (matching_study['Timepoint'].str.lower() == timepoint_input) &
                (matching_study['Screener'] == False) &
                (matching_study['Regimen'] == False)
            ]
            
        if filtered.empty:
            return None
        if question_num:
            filtered = filtered[filtered['Question_Number'].str.lower() == question_num]
        
        if not filtered.empty:
            return filtered.iloc[0]['Question']
        else:
            return None
    new_df = df_input.copy()
    new_df['Questionnaire'] = new_df.apply(match_question, axis=1)
    return new_df

In [8]:
file_path = '4146176 Raw Data.xlsx'
sheet_names = pd.ExcelFile(file_path).sheet_names
data_sheet = [sheet_name for sheet_name in sheet_names if 'map' in sheet_name.lower()][0]
df = pd.read_excel(file_path, sheet_name=data_sheet, header=None)
study_number = re.search(r"(\d{7})", file_path).group(1) if re.search(r"(\d{7})", file_path) else None
timepoints_letters = assign_timepoint_letters(res_df)
result = process_questionnaire(df,study_number,timepoints_letters)
map_new_df = merge_question_data(result,res_df)
map_new_df

,Study_Number,Variable,Label,MappedValue,MappedLabel,Timepoint,QuestionNumber,Questionnaire
0,4146176,ID,Please carefully enter the 4-Digit ID number y...,,,Cannot be Mapped,,None
1,4146176,record,Record number,,,Cannot be Mapped,,None
2,4146176,Open numeric response,,,,Cannot be Mapped,,None
3,4146176,Location,Please select your test location:,1,National Field and Focus,Cannot be Mapped,,None
4,4146176,Location,Please select your test location:,2,Creative Consumer Research,Cannot be Mapped,,None
...,...,...,...,...,...,...,...,...
1773,4146176,F6bc2,When did you typically apply the products you ...,3,In the evening only,regimen,6b,When did you typically apply the products you ...
1774,4146176,F7,How would you describe your facial skin type a...,1,Dry (no oil in t-zone; cheeks are dry and tight),regimen,7,How would you describe your facial skin type a...
1775,4146176,F7,How would you describe your facial skin type a...,2,Normal (t-zone is normal; cheeks are normal),regimen,7,How would you describe your facial skin type a...
1776,4146176,F7,How would you describe your facial skin type a...,3,Combination (t-zone is oily; cheeks are normal...,regimen,7,How would you describe your facial skin type a...


In [9]:
def process_data(raw_df, mapping_df):
    # Create a lookup from mapping_df including Label, Questionnaire, and Timepoint
    mapping_lookup = mapping_df.drop_duplicates(subset="Variable", keep="first")\
                               .set_index("Variable")[["Label", "QuestionNumber", "Questionnaire", "Timepoint"]]
    
    # Melt raw_df to long format
    df_long = raw_df.melt(id_vars="id", var_name="raw_question_number", value_name="raw_value")
    df_long.rename(columns={"id": "panelist_id"}, inplace=True)
    
    # Merge to get raw_question_meaning, Questionnaire, and Timepoint based on Variable (raw_question_number)
    df_long = df_long.merge(mapping_lookup, left_on="raw_question_number", right_index=True, how="left")
    df_long.rename(columns={"Label": "raw_question_meaning"}, inplace=True)
    
    # Prepare join key in mapping_df
    mapping_df = mapping_df.copy()
    mapping_df['MappedValue_numeric'] = pd.to_numeric(mapping_df['MappedValue'], errors='coerce')
    mapping_df['MappedValue_str'] = mapping_df['MappedValue'].astype(str).str.strip()
    mapping_df['join_key'] = np.where(
        mapping_df['MappedValue_numeric'].notnull(),
        mapping_df['Variable'] + '__' + mapping_df['MappedValue_numeric'].astype(str),
        mapping_df['Variable'] + '__' + mapping_df['MappedValue_str']
    )
    
    # Prepare join key in df_long
    df_long['raw_value_numeric'] = pd.to_numeric(df_long['raw_value'], errors='coerce')
    df_long['raw_value_str'] = df_long['raw_value'].astype(str).str.strip()
    df_long['join_key'] = np.where(
        df_long['raw_value_numeric'].notnull(),
        df_long['raw_question_number'] + '__' + df_long['raw_value_numeric'].astype(str),
        df_long['raw_question_number'] + '__' + df_long['raw_value_str']
    )
    
    # Merge mapping for join to get MappedLabel (raw_value_meaning)
    mapping_for_join = mapping_df[['join_key', 'MappedLabel']]
    df_long = df_long.merge(mapping_for_join, on='join_key', how='left')
    df_long.rename(columns={'MappedLabel': 'raw_value_meaning'}, inplace=True)
    
    # Select desired columns, including the Timepoint from mapping_lookup merge
    desired_columns = [
        "panelist_id", 
        "raw_question_number",
        "Timepoint", 
        "raw_question_meaning",
        "raw_value", 
        "raw_value_meaning",
        "QuestionNumber", 
        "Questionnaire"
    ]
    df_long = df_long[desired_columns]
    
    return df_long

In [10]:
file_path ='4146176 Raw Data.xlsx'
sheet_names = pd.ExcelFile(file_path).sheet_names
data_sheet = [sheet_name for sheet_name in sheet_names if 'map' not in sheet_name.lower() and 'label' not in sheet_name.lower()][0]
data_df = pd.read_excel(file_path, sheet_name=data_sheet)
if data_df.iloc[0].apply(lambda x: isinstance(x, str)).all():
    data_df.columns = data_df.iloc[0]
    data_df = data_df[1:]
data_df.columns = [col.lower() if 'id' in col.lower() else col for col in data_df.columns]
study_number = re.search(r"(\d{7})", file_path).group(1) if re.search(r"(\d{7})", file_path) else None

final_df = process_data(data_df,map_new_df[map_new_df['Study_Number'].str.strip() == study_number.strip()])
final_df.insert(loc=0, column='Study_Number', value=study_number)
final_df

,Study_Number,panelist_id,raw_question_number,Timepoint,raw_question_meaning,raw_value,raw_value_meaning,QuestionNumber,Questionnaire
0,4146176,1001,record,Cannot be Mapped,Record number,9.0,NaN,,None
1,4146176,1002,record,Cannot be Mapped,Record number,10.0,NaN,,None
2,4146176,1003,record,Cannot be Mapped,Record number,13.0,NaN,,None
3,4146176,1004,record,Cannot be Mapped,Record number,15.0,NaN,,None
4,4146176,1005,record,Cannot be Mapped,Record number,18.0,NaN,,None
...,...,...,...,...,...,...,...,...,...
70780,4146176,5029,F7,regimen,How would you describe your facial skin type a...,4.0,Oily (very oily in t-zone and cheeks; may have...,7,How would you describe your facial skin type a...
70781,4146176,5030,F7,regimen,How would you describe your facial skin type a...,2.0,Normal (t-zone is normal; cheeks are normal),7,How would you describe your facial skin type a...
70782,4146176,5031,F7,regimen,How would you describe your facial skin type a...,3.0,Combination (t-zone is oily; cheeks are normal...,7,How would you describe your facial skin type a...
70783,4146176,5032,F7,regimen,How would you describe your facial skin type a...,3.0,Combination (t-zone is oily; cheeks are normal...,7,How would you describe your facial skin type a...
